# Pre-Work

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Attempt 1

### Create Weight Checkpoint

In [ ]:
checkpoint = ModelCheckpoint('weights.hdf5', \
                              mode='max', \
                              monitor='val_accuracy', \
                              save_best_only=True)
callbacks_list=[checkpoint]

### Datagen

In [3]:
trainpath = '../data/raw/imgs/train'
testpath = '../data/raw/imgs/testlabeled'

In [4]:
datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
testdatagen = keras.preprocessing.image.ImageDataGenerator()

In [5]:
train = datagen.flow_from_directory(trainpath, \
                                   target_size=(480, 640), \
                                   subset='training', \
                                   seed=42)

Found 17943 images belonging to 10 classes.


In [6]:
val = datagen.flow_from_directory(trainpath, \
                                  target_size=(480, 640), \
                                  subset='validation', \
                                  seed=42)

Found 4481 images belonging to 10 classes.


In [7]:
test = datagen.flow_from_directory(testpath, \
                                   target_size=(480, 640))

Found 200 images belonging to 10 classes.


### Build Model

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(480, 640, 3)))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 478, 638, 10)      280       
Total params: 280
Trainable params: 280
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.add(MaxPool2D(30))

In [12]:
model.add(Flatten())

In [13]:
model.add(Dense(10, activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 478, 638, 10)      280       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 21, 10)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3150)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                31510     
Total params: 31,790
Trainable params: 31,790
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Fit Model

In [17]:
model.fit(train, \
          epochs=3, \
          validation_data=test, \
          callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 561 steps, validate for 7 steps
Epoch 1/3
561/561 [==============================] - 316s 564ms/step - loss: 4.9736 - accuracy: 0.6985 - val_loss: 10.3101 - val_accuracy: 0.3250
Epoch 2/3
561/561 [==============================] - 207s 369ms/step - loss: 0.2891 - accuracy: 0.9359 - val_loss: 10.8733 - val_accuracy: 0.3400
Epoch 3/3
561/561 [==============================] - 211s 377ms/step - loss: 0.1438 - accuracy: 0.9646 - val_loss: 12.3586 - val_accuracy: 0.3600


# Attempt 2

### Create Weight Checkpoint

In [47]:
checkpoint = ModelCheckpoint('weights.hdf5', \
                              mode='max', \
                              monitor='val_accuracy', \
                              save_best_only=True)
callbacks_list=[checkpoint]

### Datagen

In [48]:
trainpath = '../data/raw/imgs/train'
testpath = '../data/raw/imgs/testlabeled'

In [49]:
datagen = keras.preprocessing.image.ImageDataGenerator()
testdatagen = keras.preprocessing.image.ImageDataGenerator()

In [50]:
train = datagen.flow_from_directory(trainpath)

Found 22424 images belonging to 10 classes.


In [51]:
test = datagen.flow_from_directory(testpath)

Found 200 images belonging to 10 classes.


### Build Model

In [52]:
model = Sequential()

In [53]:
model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))

In [54]:
model.add(Conv2D(10, kernel_size=3, activation='relu'))

In [55]:
model.add(MaxPool2D(30))

In [56]:
model.add(Flatten())

In [57]:
model.add(Dense(10, activation='softmax'))

In [58]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 10)      280       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 252, 252, 10)      910       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 10)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                6410      
Total params: 7,600
Trainable params: 7,600
Non-trainable params: 0
_________________________________________________________________


### Fit Model

In [60]:
model.fit(train, \
          epochs=100, \
          steps_per_epoch=20, \
          validation_data=test, \
          callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 7 steps
Epoch 1/100
20/20 [==============================] - 7s 364ms/step - loss: 88.6667 - accuracy: 0.1297 - val_loss: 52.0143 - val_accuracy: 0.1200
Epoch 2/100
20/20 [==============================] - 7s 331ms/step - loss: 36.0103 - accuracy: 0.1203 - val_loss: 41.5302 - val_accuracy: 0.1350
Epoch 3/100
20/20 [==============================] - 6s 319ms/step - loss: 22.8517 - accuracy: 0.1453 - val_loss: 27.0028 - val_accuracy: 0.1800
Epoch 4/100
20/20 [==============================] - 7s 347ms/step - loss: 15.8309 - accuracy: 0.1531 - val_loss: 20.5478 - val_accuracy: 0.1900
Epoch 5/100
20/20 [==============================] - 6s 313ms/step - loss: 11.5506 - accuracy: 0.1922 - val_loss: 16.2754 - val_accuracy: 0.1950
Epoch 6/100
20/20 [==============================] - 7s 327ms/step - loss: 9.6441 - accuracy: 0.2109 - val_loss: 12.6224 - val_accuracy: 0.1500
Epoch 7/100
20/20 [=====================

Seems like the validation accuracy is converging around 0.40. Model is still overfitting.

# Attempt 3

### Create Weight Checkpoint

In [3]:
checkpoint = ModelCheckpoint('weights.hdf5', \
                              mode='max', \
                              monitor='val_accuracy', \
                              save_best_only=True)
callbacks_list=[checkpoint]

### Datagen

In [4]:
trainpath = '../data/raw/imgs/train'
testpath = '../data/raw/imgs/testlabeled'

In [5]:
datagen = keras.preprocessing.image.ImageDataGenerator()
testdatagen = keras.preprocessing.image.ImageDataGenerator()

In [6]:
train = datagen.flow_from_directory(trainpath)

Found 22424 images belonging to 10 classes.


In [7]:
test = datagen.flow_from_directory(testpath)

Found 200 images belonging to 10 classes.


### Build Model

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))

Add BatchNormalization()

In [10]:
model.add(BatchNormalization())

In [11]:
model.add(Conv2D(10, kernel_size=3, activation='relu'))

In [12]:
model.add(MaxPool2D(30))

In [13]:
model.add(Flatten())

In [14]:
model.add(Dense(10, activation='softmax'))

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 10)      280       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 10)      40        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 10)      910       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 10)          0         
_________________________________________________________________
flatten (Flatten)            (None, 640)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                6410      
Total params: 7,640
Trainable params: 7,620
Non-trainable params: 20
_____________________________________________________

### Fit Model

In [17]:
model.fit(train, \
          epochs=50, \
          steps_per_epoch=20, \
          validation_data=test, \
          callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 7 steps
Epoch 1/50
20/20 [==============================] - 9s 435ms/step - loss: 5.4399 - accuracy: 0.1125 - val_loss: 8.7437 - val_accuracy: 0.1150
Epoch 2/50
20/20 [==============================] - 6s 316ms/step - loss: 2.9363 - accuracy: 0.1531 - val_loss: 4.9533 - val_accuracy: 0.1050
Epoch 3/50
20/20 [==============================] - 6s 300ms/step - loss: 2.3498 - accuracy: 0.2031 - val_loss: 3.3772 - val_accuracy: 0.1300
Epoch 4/50
20/20 [==============================] - 6s 323ms/step - loss: 2.2304 - accuracy: 0.2156 - val_loss: 3.0705 - val_accuracy: 0.1400
Epoch 5/50
20/20 [==============================] - 6s 310ms/step - loss: 2.1774 - accuracy: 0.2719 - val_loss: 2.8172 - val_accuracy: 0.1800
Epoch 6/50
20/20 [==============================] - 6s 309ms/step - loss: 2.0685 - accuracy: 0.2937 - val_loss: 2.6179 - val_accuracy: 0.1800
Epoch 7/50
20/20 [==============================] - 6s 28

# Attempt 4

### Create Weight Checkpoint

In [18]:
checkpoint = ModelCheckpoint('weights.hdf5', \
                              mode='max', \
                              monitor='val_accuracy', \
                              save_best_only=True)
callbacks_list=[checkpoint]

### Datagen

In [19]:
trainpath = '../data/raw/imgs/train'
testpath = '../data/raw/imgs/testlabeled'

In [20]:
datagen = keras.preprocessing.image.ImageDataGenerator()
testdatagen = keras.preprocessing.image.ImageDataGenerator()

In [21]:
train = datagen.flow_from_directory(trainpath)

Found 22424 images belonging to 10 classes.


In [22]:
test = datagen.flow_from_directory(testpath)

Found 200 images belonging to 10 classes.


### Build Model

In [23]:
model = Sequential()

In [24]:
model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))

Add Dropout layer

In [25]:
model.add(Dropout(0.25))

In [26]:
model.add(Conv2D(10, kernel_size=3, activation='relu'))

In [27]:
model.add(MaxPool2D(30))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(10, activation='softmax'))

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 10)      280       
_________________________________________________________________
dropout (Dropout)            (None, 254, 254, 10)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 252, 252, 10)      910       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 10)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6410      
Total params: 7,600
Trainable params: 7,600
Non-trainable params: 0
____________________________________________________

### Fit Model

In [32]:
model.fit(train, \
          epochs=50, \
          steps_per_epoch=20, \
          validation_data=test, \
          callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 7 steps
Epoch 1/50
20/20 [==============================] - 6s 317ms/step - loss: 115.0808 - accuracy: 0.1109 - val_loss: 48.1421 - val_accuracy: 0.0800
Epoch 2/50
20/20 [==============================] - 5s 272ms/step - loss: 36.6576 - accuracy: 0.1094 - val_loss: 29.9051 - val_accuracy: 0.1000
Epoch 3/50
20/20 [==============================] - 5s 273ms/step - loss: 22.0444 - accuracy: 0.1281 - val_loss: 22.2663 - val_accuracy: 0.1000
Epoch 4/50
20/20 [==============================] - 5s 273ms/step - loss: 16.3353 - accuracy: 0.1266 - val_loss: 18.3861 - val_accuracy: 0.1250
Epoch 5/50
20/20 [==============================] - 5s 273ms/step - loss: 13.8963 - accuracy: 0.1531 - val_loss: 15.0430 - val_accuracy: 0.1300
Epoch 6/50
20/20 [==============================] - 5s 273ms/step - loss: 10.6929 - accuracy: 0.1719 - val_loss: 13.0865 - val_accuracy: 0.1450
Epoch 7/50
20/20 [==========================